# Heart attack prediction

In [35]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 736.2 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 1.6 MB/s eta 0:00:0000:0100:03


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import numpy as np

from xgboost import XGBClassifier
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [23]:
# 1. Charger le fichier train.csv

ex = pd.read_csv('extra.csv')
train = pd.read_csv('train.csv')

train_data = pd.concat([ex, train])

# Colonnes d'intérêt
columns = ["_RFHLTH", "_PHYS14D", "_MENT14D", "_HLTHPLN", "_TOTINDA", "_EXTETH3", "_ALTETH3", "_DENVST3", "_LTASTH1", 
           "_CASTHM1", "_ASTHMS1", "_DRDXAR2", "_RACEPR1", "_SEX", "_AGEG5YR", "TARGET"]

# Extraction des colonnes
train_data = train_data[columns]
print(train_data)

        _RFHLTH  _PHYS14D  _MENT14D  _HLTHPLN  _TOTINDA  _EXTETH3  _ALTETH3  \
0           1.0       2.0       2.0       1.0       1.0       2.0       1.0   
1           2.0       3.0       3.0       1.0       1.0       1.0       NaN   
2           1.0       1.0       1.0       1.0       1.0       1.0       NaN   
3           1.0       1.0       1.0       1.0       1.0       2.0       NaN   
4           1.0       1.0       2.0       1.0       2.0       1.0       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
224995      1.0       1.0       1.0       1.0       1.0       1.0       1.0   
224996      1.0       3.0       3.0       1.0       2.0       2.0       NaN   
224997      1.0       1.0       1.0       1.0       1.0       1.0       NaN   
224998      1.0       2.0       1.0       1.0       1.0       9.0       NaN   
224999      1.0       1.0       1.0       1.0       1.0       2.0       1.0   

        _DENVST3  _LTASTH1  _CASTHM1  _ASTHMS1  _DR

In [14]:
# 2. Nettoyage des données : remplacer les NaN par la moyenne (ou une autre stratégie)
imputer = SimpleImputer(strategy='mean')  # Utiliser la moyenne pour les valeurs manquantes
train_data_cleaned = pd.DataFrame(imputer.fit_transform(train_data), columns=columns)

# Séparer les données en variables explicatives (X) et cible (y)
X = train_data_cleaned.drop("TARGET", axis=1)
y = train_data_cleaned["TARGET"]

# Diviser en ensemble d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# 3. Entraîner un modèle (Random Forest Classifier)
model = RandomForestClassifier(n_estimators=200,random_state=42)
model.fit(X_train, y_train)

# Évaluer le modèle sur l'ensemble de validation
y_pred = model.predict(X_val)
accuracy = f1_score(y_val, y_pred)
print(f"Accuracy sur l'ensemble de validation: {accuracy:.2f}")

Accuracy sur l'ensemble de validation: 0.13


In [32]:
# 4. Charger le fichier test.csv

test_data = pd.read_csv('test.csv')

# Conserver uniquement les colonnes d'intérêt
columns_test = ["_RFHLTH", "_PHYS14D", "_MENT14D", "_HLTHPLN", "_TOTINDA", "_EXTETH3", "_ALTETH3", "_DENVST3", "_LTASTH1", 
           "_CASTHM1", "_ASTHMS1", "_DRDXAR2", "_RACEPR1", "_SEX", "_AGEG5YR", "ID"]
test_data = test_data[columns_test]

# Nettoyer les données (remplacer les NaN)
test_data_cleaned = test_data.drop("ID", axis=1)
imputer = SimpleImputer(strategy='mean')  # Utiliser la moyenne pour les valeurs manquantes
test_data_cleaned = pd.DataFrame(imputer.fit_transform(test_data_cleaned), columns=columns_test[:-1])

In [33]:
# 5. Effectuer les prédictions
test_predictions = model.predict(test_data_cleaned)

# Ajouter les prédictions au dataframe original
test_data["TARGET"] = test_predictions

# Exporter les résultats dans un fichier prediction.csv
output_file = 'prediction.csv'
test_data[["ID", "TARGET"]].to_csv(output_file, index=False)

print(f"Les prédictions ont été enregistrées dans le fichier {output_file}.")

Les prédictions ont été enregistrées dans le fichier prediction.csv.
